In [118]:
import os 
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


In [2]:
os.getcwd()

'c:\\TCGA-LIHC'

Data Loading

In [3]:
expr_data = pd.read_table("./gene_expression")
gene_list = expr_data['sample']
expr = expr_data.drop(['sample'],axis=1)
expr = expr.transpose()

clinical_data = pd.read_table("./TCGA.LIHC.sampleMap_LIHC_clinicalMatrix")
idx = []
for sample in expr.index:
    sampleID_list = clinical_data['sampleID'].tolist()
    tmp = sampleID_list.index(sample)
    idx.append(tmp)

clinical_core = clinical_data.reindex(idx)
map_dict = {'Primary Tumor' : 1,
            'Recurrent Tumor' : 1,
            'Solid Tissue Normal' : 0}
clinical_core = clinical_core['sample_type'].apply(lambda x : map_dict[x])

In [122]:
X = torch.tensor(expr.to_numpy(), dtype=torch.float)
y = torch.tensor(clinical_core.to_numpy(), dtype=torch.long)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

torch.Size([296, 20530]) torch.Size([127, 20530]) torch.Size([296]) torch.Size([127])


In [123]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

class TrainDataset(Dataset):
    def __init__(self):
        self.x_data = train_X
        self.y_data = train_y
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, index):
        x = torch.tensor(self.x_data[index]).clone().detach()
        y = torch.tensor(self.y_data[index]).clone().detach()
        return x, y
    
class TestDataset(Dataset):
    def __init__(self):
        self.x_data = test_X
        self.y_data = test_y
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, index):
        x = torch.tensor(self.x_data[index]).clone().detach()
        y = torch.tensor(self.y_data[index]).clone().detach()
        return x, y
    
training_data = TrainDataset()
test_data = TestDataset()

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

Neural Network

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20530, 1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,2),
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [146]:
model = NeuralNetwork()

learning_rate = 3e-5
batch_size = 64
epochs = 20

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()   # bc/ gradients by default add up; to prevent double counting.
        loss.backward() # deposits gradients
        optimizer.step()    # adjust parameters with gradients

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# iterate over test dataset to check model performance
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [147]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 0.923781  [    0/  296]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.715607 

Epoch 2
-------------------------------
loss: 0.407823  [    0/  296]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.294592 

Epoch 3
-------------------------------
loss: 0.213487  [    0/  296]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.352891 

Epoch 4
-------------------------------
loss: 0.207963  [    0/  296]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.261639 

Epoch 5
-------------------------------
loss: 0.232299  [    0/  296]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.245034 

Epoch 6
-------------------------------
loss: 0.152441  [    0/  296]
Test Error: 
 Accuracy: 96.1%, Avg loss: 0.196287 

Epoch 7
-------------------------------
loss: 0.179055  [    0/  296]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.163821 

Epoch 8
-------------------------------
loss: 0.113939  [    0/  296]
Test Error: 
 Accuracy: 96.1%, Avg loss: 0.137807 

Epoch 9
----------------